## Homework 1: ImageGPT

Мы привыкли, что для работы с изображениями нужны сверточные сети. Но что, если взглянуть на задачу под другим углом? В этой работе мы отойдем от классического подхода и исследуем, как можно применить архитектуру **Трансформер** для генерации изображений. Для этого мы реализуем архитектуру **ImageGPT**, которая обрабатывает изображения так же, как классический GPT работает с текстом - как с последовательностью токенов.

### Основная идея
Эта идея впервые подробно была описана в работе [Generative Pretraining from Pixels](https://cdn.openai.com/papers/Generative_Pretraining_from_Pixels_V2.pdf) от **OpenAI**. Её авторы показали, что если представить изображение в виде последовательности, то модель способна обучиться не хуже, чем классические сверточные сети. 

Суть этой идеи в том, что вместо сверточных слоёв **ImageGPT** преобразует изображение в последовательность квантованных пикселей, которые выступают в роли токенов. Эта последовательность затем подаётся на вход GPT, и модель учится предсказывать следующий токен.

### Задание
Вам предстоит реализовать основные блоки архитектуры ImageGPT и обучить модель на датасете `CIFAR-10` для решения задачи генерации изображений.

За выполнение домашнего задания можно получить до **10 баллов**. Для части заданий мы написали для вас скелет. Заполните в них пропуски, выделенные с помощью `...`

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from IPython.display import clear_output
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Задание 1: Dataset (0.5 балла)

Для обучения нашей модели мы будем использовать популярный датасет `CIFAR-10`. Он состоит из $60 000$ цветных изображений размером $32\times 32$ пикселя, разделенных на $10$ классов.

Датасет уже за вас поделен на **50 000** обучающих и **10 000** тестовых изображений.

Для удобства дальнейшей работы нам нужно получить датасет в формате Dataset из PyTorch. Проще всего получить из библиотеки `torchvision.datasets`.

In [ ]:
from torchvision.datasets import CIFAR10

- Создайте преобразования `transforms` с аугментациями `RandomCrop` и `RandomHorizontalFlip` для обучающего (`train_dataset`) и без аугментации для валидационного (`val_dataset`) датасетов.
- Загрузите CIFAR-10 с соответствующими преобразованиями
- Создайте DataLoaders для каждого датасета

Прежде чем приступить к построению модели всегда полезно взглянуть на данные, с которыми предстоит работать. Это даёт общее представление о данных и позволяет убедиться, что всё загрузилось корректно.

In [ ]:
mappings = {
    0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 
    5: 'dog', 6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'
}

# Select a random set of indices from the training dataset.
indices = torch.randperm(len(train_dataset))[:40]

fig, axes = plt.subplots(5, 8, figsize=(16, 10))

for i, ax in enumerate(axes.flat):
    image, label = train_dataset[indices[i]]  

    image = image.permute(1, 2, 0).cpu().numpy()

    ax.imshow(image)
    ax.axis('off')
    ax.set_title(f"{mappings[label]}")

plt.tight_layout()
plt.show()

### Задание 2: Квантование изображений (1.5 балла)

Теперь давайте разберёмся, как нам подготовить изображения для трансформера.  

Трансформеры работают с **дискретными токенами**, но стандартные цифровые изображения обычно хранят значения пикселей в виде дискретных целых чисел от 0 до 255. То есть каждый пискель может иметь одно из $256^3\approx 16.8$ миллиона возможных цветовых комбинаций. Если бы мы использовали каждую из этих комбинаций как отдельный токен для нашей модели, то у нас был бы просто огромный словарь, и вряд ли бы мы смогли обучить трансформер на таких данных.

Решение этой проблемы лежит в квантовании. Обычно в PyTorch мы используем преобразование `torchvision.transforms.ToTensor()`. Оно не только конвертирует изображение в тензор PyTorch, но и нормализует значения пикселей из диапазона $[0, 255]$ в диапазон $[0.0, 1.0]$. В итоге, наши данные становятся непрерывными.

Чтобы сделать их дискретными, в ImageGPT предложили метод квантования пикселей с помощью `K-Means`. Вместо того чтобы представлять каждый пиксель его `float` значением, мы группируем похожие цвета в небольшое, заранее заданное количество **цветовых кластеров** или **кодовых слов** (**codebook**). Каждый пиксель изображения затем заменяется целочисленным индексом того цветового кластера, к которому он наиболее близок.

Кратко процесс выглядит так:

1) **Собираем все пиксели из нашего датасета**.

2) **Применяем алгоритм K-Means к этим пикселям с желаемым количеством кластеров**.

В результате мы получаем $K$ кластеров. Каждый из них представляет собой усреднённый цвет (центроид) для своей группы пикселей. Эти центроиды и станут нашим словарём для дальнейшего квантования изображений.

Теперь давайте обучим `K-Means` на наших данных с `n_clusters=512` и сохраним кластеры в формате `.npy`.

In [ ]:
def create_cifar10_centroids(n_clusters=512, save_path="path/to/save"):
    
    print("Loading CIFAR-10...")
    dataset = ... # TODO: pay attention to transforms

    print("Extracting pixels...")
    all_pixels = []
    
    for img, _ in dataset:
        img_np = ...
        all_pixels.append(...) 
        
    all_pixels = np.concatenate(all_pixels, axis=0)

    print(f"Clustering with KMeans (n_clusters={n_clusters})...")
    kmeans = ...
    centroids = kmeans.cluster_centers_

    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    np.save(save_path, centroids)
    print(f"Centroids saved to {save_path}")

In [ ]:
create_cifar10_centroids(n_clusters=512, save_path="clusters/cifar10_clusters.npy")

Также во время обучения нам понадобятся еще несколько вспомогательных функций:

- **`img_to_seq`**: преобразует двумерное изображение в одномерную последовательность токенов. Тут используем формат сначала размер последовательности, потом размер батча.

- **`quantize`**: принимает на вход изображение и преобразует его в изображение, где каждый пиксель заменен индексом ближайшего центроида из нашего словаря.

- **`unquantize`**: выполняет обратную операцию, которая принимает квантованное изображение и восстанавливает его в пиксельное представление

In [ ]:
def img_to_seq(x):
    # Reshape x to [Batch_Size, H*W] and then transpose to [H*W, Batch_Size]
    # Use .contiguous() to avoid view() and inplace-ops errors after transpose.
    x = ...
    return x

def quantize(x, centroids):
    b, c, h, w = x.shape
    
    # Calculate Euclidean distance using (a-b)^2 = a^2 - 2ab + b^2 
    # and find the index of the closest centroid for each pixel
   
    return x.view(b, h, w)      

def unquantize(x, centroids):
    
    return ... 

### Задание 3: Embeddings (1.5 балла)

Чтобы трансформер мог работать с изображениями, их нужно преобразовать в понятный для него формат. В текстовых моделях слова сначала превращаются в  токены, а затем — в вектора эмбеддингов, в которых хранится "смысл слов".

С изображениями мы поступаем так же, но вместо слов мы работаем с пикселями. Мы уже отметили, что каждый пиксель в ImageGPT рассматривается как отдельный токен, который получает индекс из нашего словаря, созданного с помощью K-Means. Однако трансформеру нужны не просто эти индексы, а их векторные представления, которые он может обрабатывать. В процессе обучения эти векторы постепенно учатся отражать некоторые связи между пикселями, что позволяет модели понимать изображение.

В ImageGPT нам потребуются несколько типов эмбеддингов для кодирования входных изображений:

- **`token_embeddings`**: преобразуют индексы квантованных пикселей в вектора

- **`position_embeddings`**: кодируют информацию о позиции каждого пикселя, это позволяет модели улавливать пространственные отношения между ними

- **`class_embedding`**: преобразует индекс класса в вектор и выступает в роли условного стартового токена, который сообщает модели, изображение какого именно класса нужно генерировать

In [ ]:
class GPTEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_positions, num_classes):
        super().__init__()
        self.token_embeddings = ...
        self.position_embeddings = ...
        self.class_embedding = ...

    def forward(self, x, cls_label):
        
        tok_emb = ...
        cls_token_emb = ...

        full_seq = ... # concat cls_token

        pos_ids = ...
        pos_emb = ...


        # Return final embedding sequence:
        # Add positional information to full_seq
        
        return ...

### Задание 5: Decoder Block (1.5 балла)

Теперь нам нужно собрать основные блоки нашей модели — блоки декодера. В каждом таком блоке последовательно будут выполняются следующие операции:

- **`LayerNorm`**
- **`MultiHeadAttention`**
- **`Residual Connection`**
- **`LayerNorm`**
- **`MLP`**:
  - **Linear**: $embed\; dim$ -> $4 \times embed\; dim$
  - **GELU Activation**
  - **Linear**: $(4 \times embed\; dim)$ -> $embed\; dim$
- **`Residual Connection`**

Поскольку наша модель является генеративной и предсказывает следующий элемент в последовательности, вам потребуется использовать **Masked Self-Attenion**.

Вы можете вопользоваться готовой реализацией **Multi-Head Attention** из Pytroch.

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        self.ln1 = ...
        self.attn = ...  
        self.ln2 = ...  
        self.mlp = nn.Sequential(
            ... ,  
            ... ,  
            ...  
        )

    def forward(self, x):

        return ...

### Задание 6: Final Model (1 балл)

Теперь, когда у нас есть все компоненты, мы можем собрать их вместе, чтобы построить итоговую модель ImageGPT.

Итоговая архитектура выглядит так:

- **`Embedding`**: преобразует входные токены в векторы

- **`Blocks`**: использует `num_layers` блоков `DecoderBlock`, которые последовательно обрабатывают наши данные

- **`Final LayerNorm`**: нормализует выходные данные перед финальным предсказанием

- **`Head`**: преобразует обработанные векторы в предсказания

- **`Centroids`**: храним центроиды как параметр модели, они необходимы для обратного преобразования токенов в реальные цвета.

In [ ]:
class ImageGPT(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, num_layers, max_positions, centroids_path, num_classes):
        super().__init__()
        self.embedding = ...
        self.blocks = ...
        self.ln_f = ...
        self.head = ...

        centroids = torch.tensor(np.load(centroids_path), dtype=torch.float32)
        self.register_buffer('centroids', centroids)

    def forward(self, x, y):        
        # Compute embeddings
        h = ...  
        
        for block in self.blocks:
            h = ...  
        
        # Apply final LayerNorm
        h = ...  
        
        # Project to vocabulary logits
        output = ...
        
        return output

In [ ]:
centroids_path="clusters/cifar10_clusters.npy"

model = ImageGPT(
    centroids_path=centroids_path,
    embed_dim=512,
    num_heads=8,
    num_layers=24,
    max_positions=32*32,
    vocab_size=512,
    num_classes=10
    )

total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

### Задание 7: Sampling (1.5 балла)

Для генерации новых изображений мы будем использовать уже обученную модель ImageGPT. Давайте напишем функцию **`sample`**, которая будет отвечать за это.

Она будет принимать на вход следующие параметры:

- **`model`**: наша обученная модель ImageGPT.

- **`length`**: число шагов генерации

- **`context`**: начальная последовательность пикселей, если мы хотим дорисовать изображение. Если её нет, генерация начнётся с нуля.

- **`class_label`**: метка класса для генерации по заданному условию

- **`temperature`**: параметр, который регулирует уровень случайности при генерации

- **`num_samples`**: количество изображений, которые нужно сгенерировать

In [ ]:
def sample(model, length, class_label, num_samples=1, device='cuda',
           context=None, temperature=1.0):

    # expand it so that each generated image has a class label
    class_label = ...
    
    if context is None or context.size(0) == 0:
        context = torch.empty((0, num_samples), dtype=torch.long, device=device)
    else:
        if context.dim() == 1:
            # Add a new dimension to make it compatible with the model's input format.
            ...
        # Repeat the context to generate multiple images from the same starting sequence

    with torch.no_grad():
        # Implement the generation loop here
        ...

    return ...

### Задание 8: Training and Validation Loop (1.5 балла)

После того как все компоненты модели собраны, мы можем приступить к обучению. Здесь вамм нужно реализовать основные функции для тренировки и валидации нашей модели.

In [ ]:
def train_one_epoch(model, train_loader, optimizer, criterion, scheduler, device):
    
    model.train()  
    total_loss = 0.0

    for images, labels in tqdm(train_loader, desc="Train"):
        images, labels = ...  

        logits = ...  

        loss = ...  

        total_loss += ...  

    # Compute and return average loss for this epoch
    
    return ...  


def validate_one_epoch(model, val_loader, criterion, device):

    model.eval()  
    total_loss = 0.0

    with torch.no_grad():  
        for images, labels in tqdm(val_loader, desc="Val"):
            images, labels = ...  

            logits = ... 

            loss = ...  
            total_loss += ... 

    # Compute and return average loss for this epoch
    
    return ...

def save_checkpoint(model, epoch, path_template):
    save_path = path_template.format(epoch=epoch+1)
    save_dir = os.path.dirname(save_path)
    if save_dir:  
        os.makedirs(save_dir, exist_ok=True)
        
    torch.save(model.state_dict(), save_path)
    print(f"Checkpoint saved: {save_path}")

def plot_losses(train_losses, val_losses):
    clear_output()
    plt.figure(figsize=(6, 4))
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.title("Training and Validation Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

Для визуализации результатов работы нашей модели нам потребуется несколько вспомогательных функций. Ниже представлены некоторые функции, которые помогут нам подготовить данные, запустить процесс генерации и сравнить сгенерированные изображения с оригиналом.

In [ ]:
def pick_val_sample(val_loader, device):
    img_sample, class_sample = next(iter(val_loader))
    img_sample = img_sample[0:1].to(device)
    class_sample = class_sample[0:1].to(device)
    return img_sample, class_sample

def generate_image_tokens(model, class_sample, h, w, device):
    num_tokens = h * w
    gen_seq = sample(
        model,
        context=None,
        class_label=class_sample,
        length=num_tokens,
        device=device
    ).cpu().numpy().squeeze()
    return gen_seq

def image_to_tokens(image_tensor, centroids):
    token_indices = quantize(image_tensor, centroids).cpu().numpy()
    return token_indices.reshape(-1)

def tokens_to_image(tokens, h, w, centroids):
    tokens_tensor = torch.tensor(tokens, dtype=torch.long).reshape(1, h, w)
    rgb_image = unquantize(tokens_tensor, centroids).cpu().numpy().squeeze()
    rgb_image = (rgb_image * 255).astype(np.uint8)
    return rgb_image

def plot_generated_vs_original(gen_img_rgb, original_img_rgb, class_label):
    final_img = np.concatenate([gen_img_rgb, original_img_rgb], axis=1)
    plt.figure(figsize=(4, 2))
    plt.title(f"Generated vs Original – Class {class_label}")
    plt.imshow(final_img)
    plt.axis("off")
    plt.show()

def generate_and_plot_sample(model, val_loader, device):
    model.eval()
    centroids = model.centroids
    img_sample, class_sample = pick_val_sample(val_loader, device)
    h, w = img_sample.shape[-2:]

    gen_seq = generate_image_tokens(model, class_sample, h, w, device)
    generated_image_rgb = tokens_to_image(gen_seq, h, w, centroids)
    original_tokens = image_to_tokens(img_sample, centroids)
    original_image_rgb = tokens_to_image(original_tokens, h, w, centroids)
    
    plot_generated_vs_original(generated_image_rgb, original_image_rgb, class_sample.item())

In [ ]:
def train_imagegpt(model, train_loader, val_loader, optimizer, criterion,
                   scheduler, checkpoint_path, device=device, epochs=10):

    model.to(device)

    train_loss_history, val_loss_history = [], []

    for epoch in range(epochs):

        train_loss = ...
        val_loss = ...

        print(f"\nEpoch {epoch+1}: Train Loss = {train_loss:.4f} | Val Loss = {val_loss:.4f}")

        plot_losses(train_loss_history, val_loss_history)
        save_checkpoint(model, epoch, f"{checkpoint_path}/imagegpt_epoch{epoch}.pt")
        generate_and_plot_sample(model, val_loader, device)

Теперь мы готовы запустить основной цикл обучения. Вы можете экспериментировать с различными гиперпараметрами и расписанием обучения, чтобы добиться хороших результатов.

In [ ]:
epochs = 10
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs * len(train_loader))
checkpoint_path = "checkpoints_imagegpt"

train_imagegpt(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    optimizer=optimizer,
    criterion=criterion,
    scheduler=scheduler,
    checkpoint_path=checkpoint_path,
    device=device,
    epochs=epochs
)

### Задание 9: Autocompletion (1 балл)

Ниже реализованы несколько функций для удобной отрисовки результатов. Используйте их, чтобы выполнить задание.

In [ ]:
def pick_val_contexts(val_loader, num_contexts, device):
    context_images, class_labels = [], []
    with torch.no_grad():
        for images, batch_labels in val_loader:
            for img, label in zip(images, batch_labels):
                context_images.append(img.to(device))
                class_labels.append(label.to(device))
                if len(context_images) >= num_contexts:
                    return context_images, class_labels
    return context_images, class_labels

def prepare_context(img_tensor, context_size, centroids):
    original_tokens = image_to_tokens(img_tensor.unsqueeze(0), centroids).squeeze()
    context_tokens = original_tokens[:context_size]
    context_tensor = torch.tensor(context_tokens, dtype=torch.long, device=img_tensor.device)
    return context_tensor, original_tokens

def generate_variations(model, context_tensor, class_label, total_tokens, context_size, samples_per_context, temperature=1.0):
    generated_images = []
    h = w = int(total_tokens**0.5)
    centroids = model.centroids
    device = context_tensor.device

    for _ in tqdm(range(samples_per_context), desc="Generation", leave=False):
        gen_seq = sample(model, 
                         length=total_tokens - context_size, 
                         context=context_tensor,
                         class_label=class_label, 
                         temperature=temperature, 
                         device=device)
        
        gen_img_rgb = tokens_to_image(gen_seq.cpu().numpy().squeeze(), h, w, centroids)
        generated_images.append(gen_img_rgb)
    return generated_images

def plot_context_rows(rows_of_images, class_labels):
    num_contexts = len(rows_of_images)
    if num_contexts == 0:
        return
    num_cols = len(rows_of_images[0])
    fig, axs = plt.subplots(nrows=num_contexts, ncols=1, figsize=(num_cols * 2, num_contexts * 2.2))
    if num_contexts == 1: 
        axs = [axs]

    for i, (row, label) in enumerate(zip(rows_of_images, class_labels)):
        combined_row_img = np.concatenate(row, axis=1)
        axs[i].imshow(combined_row_img)
        axs[i].set_title(f"Class: {label}")
        axs[i].axis("off")
    
    plt.tight_layout()
    plt.show()

In [ ]:
def generate_and_plot_variations(model, val_loader, num_contexts=5, samples_per_context=5, context_size=512, temperature=1.0, device='cuda'):
    
    model.eval()
    centroids = model.centroids
    
    context_images, class_labels = pick_val_contexts(val_loader, num_contexts, device)
    h, w = context_images[0].shape[-2:]
    total_tokens = h * w
    all_rows_for_plotting = []
    plot_labels = [label.item() for label in class_labels]

    for img, label in tqdm(zip(context_images, class_labels), total=num_contexts):
        context_tensor, original_tokens = prepare_context(img, context_size, centroids)
        generated_imgs_rgb = generate_variations(
            model, context_tensor, label.unsqueeze(0), total_tokens, 
            context_size, samples_per_context, temperature=temperature
        )
        context_display_tokens = np.pad(original_tokens[:context_size], 
                                        (0, total_tokens - context_size), 
                                        'constant', constant_values=0)
        context_img_rgb = tokens_to_image(context_display_tokens, h, w, centroids)
        original_img_rgb = tokens_to_image(original_tokens, h, w, centroids)
        row = [context_img_rgb] + generated_imgs_rgb + [original_img_rgb]
        all_rows_for_plotting.append(row)

    plot_context_rows(all_rows_for_plotting, plot_labels)

После того как мы обучили модель, мы можем загрузить её, чтобы использовать для генерации. Код ниже загружает веса модели, которые были сохранены во время обучения.

In [ ]:
checkpoint_path = "checkpoints_imagegpt/imagegpt_epoch10.pt"

model.to(device)
model.load_state_dict(torch.load(checkpoint_path, map_location=device))
model.eval()
print(f"Model loaded from: {checkpoint_path}")

Теперь давайте перейдем к исследованию способности модели к автодополнению. Ваша задача — проверить, насколько хорошо модель может дорисовывать изображения в зависимости от объёма контекста.

Запустить функцию `generate_and_plot_variations` три раза с разными значениями `context_size = {256, 512, 768}`.

Обратите внимание, как меняется качество сгенерированных изображений в зависимости от того, сколько контекста получила модель, и сделайте выводы.

Теперь давайте исследуем влияние температуры на качество и разнообразие изображений. Этот параметр, как мы уже говорили, контролирует "случайность" предсказаний модели. 

Запустите функцию `generate_and_plot_variations` три раза, используя одинаковый размер контекста `context_size=512`, но с разными значениями температуры `temperature = {0.9, 0.7, 0.5}`. Посмотрите на результаты и сделайте выводы.

In [ ]:
# Здесь можно оставить отзывы, пожелания и впечатления о ДЗ :)